In [5]:
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import re
import os
from dotenv import load_dotenv

In [6]:
# Load YouTube API key
load_dotenv()

api_key = os.getenv("API_KEY")

# Initialize YouTube API client
youtube = build('youtube', 'v3', developerKey=api_key)

In [10]:
# Extract the channel ID's from the channel names
def get_channel_ids(channel_names):
    channel_data = []
    
    for channel_name in channel_names:
        request = youtube.search().list(
            q=channel_name,
            type='channel',
            part='id',
            maxResults=1
        )
        response = request.execute()
        
        if 'items' in response:
            channel_id = response['items'][0]['id']['channelId']
            channel_data.append({'channel_name': channel_name, 'channel_id': channel_id})
        else:
            print(f"Could not find channel ID for '{channel_name}'")
    
    df_channels = pd.DataFrame(channel_data)
    return df_channels

In [11]:
channel_names = [
    "TheArmbandFPL",
    "elitefpl",
    "fantasyfootballfixYT",
    "FFScout_",
    "AboveAverageFPL",
    "fplbanger",
    "fplblackbox",
    "FPLFocal"
]

channel_ids = df_channels = get_channel_ids(channel_names)

print(channel_ids)

           channel_name                channel_id
0         TheArmbandFPL  UC4UdmU9tNnU5iQVmQB3Ngvg
1              elitefpl  UCOhHIQyQg4dNKvWg0tg12zg
2  fantasyfootballfixYT  UC0Oaf88gRGnNkncI8D_GO-Q
3              FFScout_  UCKxYKQ8pgJ7V8wwh4hLsSXQ
4       AboveAverageFPL  UCnaJiRMf5hju0TlaeGK5CDQ
5             fplbanger  UC1dzUZYYluvh8ktUYFYk8PA
6           fplblackbox  UCGJ8-xqhOLwyJNuPMsVoQWQ
7              FPLFocal  UC72QokPHXQ9r98ROfNZmaDw


In [12]:
# Function to get the most recent video IDs from a channel
def get_channel_videos(df_channels, published_after, max_results=10):
    all_videos = []
    
    for _, row in df_channels.iterrows():
        channel_id = row['channel_id']
        channel_name = row['channel_name']
        videos = []
        next_page_token = None
        
        while True:
            request = youtube.search().list(
                part="id,snippet",
                channelId=channel_id,
                type="video",
                order="date",
                publishedAfter=published_after,
                maxResults=max_results,
                pageToken=next_page_token
            )
            response = request.execute()
            
            for item in response['items']:
                video_id = item['id']['videoId']
                title = item['snippet']['title']
                published_at = item['snippet']['publishedAt']
                videos.append({
                    'channel_name': channel_name,
                    'id': video_id,
                    'title': title,
                    'published_at': published_at
                })
                
                if len(videos) >= max_results:
                    break
            
            if len(videos) >= max_results:
                break
            
            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break
        
        all_videos.extend(videos)
    
    return pd.DataFrame(all_videos)


In [13]:
# Extract the last n video ID's for each of the youtube accounts
published_after = "2023-08-19T00:00:00Z" 
df_videos = get_channel_videos(df_channels, published_after, max_results=5)

print(df_videos)

            channel_name           id  \
0          TheArmbandFPL  hlWD3N7iohY   
1          TheArmbandFPL  RtkI_Txs8zk   
2          TheArmbandFPL  dAZs-gQkk8A   
3          TheArmbandFPL  xQVDXFka6e8   
4          TheArmbandFPL  cFRk2bdT_ik   
5               elitefpl  PdUZT6ctqmI   
6               elitefpl  c1_CUY_cQ3I   
7               elitefpl  czfSplbnOjw   
8               elitefpl  ELqFLS5AVl8   
9               elitefpl  FQTNCrxz6DI   
10  fantasyfootballfixYT  lTRqo47icdY   
11  fantasyfootballfixYT  pcQvJN43yls   
12  fantasyfootballfixYT  z6_cerpMLhQ   
13  fantasyfootballfixYT  jUlug2HTj8Q   
14  fantasyfootballfixYT  PPcwL5vXcGs   
15              FFScout_  TzIm6ecANW8   
16              FFScout_  SwlXs5B-JPA   
17              FFScout_  xX-8dQOjvS0   
18              FFScout_  vDEZYXQDQWg   
19              FFScout_  RYEuxvWfAio   
20       AboveAverageFPL  NoiJHbJhm1U   
21       AboveAverageFPL  K3V2jIT4D2c   
22       AboveAverageFPL  lqsOcdFef14   
23       AboveAv

In [17]:
# Function to grab the transcripts if they exist
def get_transcripts(df_videos):
    transcripts = []
    for _, row in df_videos.iterrows():
        video_id = row['id']
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            full_transcript = ' '.join([entry['text'] for entry in transcript])
            transcripts.append(full_transcript)
        except Exception as e:
            transcripts.append(None)
            print(f"Error getting transcript for video {video_id}: {str(e)}")
    return transcripts

# Function to calculate word count
def word_count(transcript):
    if transcript:
        return len(transcript.split())  # Split by whitespace to count words
    return 0

In [18]:
# Fetch transcripts for df_videos
transcripts = get_transcripts(df_videos)

# Add transcripts to the DataFrame
df_videos['transcript'] = transcripts

# Add word count of the transcripts
df_videos['word_count'] = df_videos['transcript'].apply(word_count)

# Display the first few rows to verify
print(df_videos[['channel_name', 'id', 'title', 'word_count', 'transcript']])

Error getting transcript for video PdUZT6ctqmI: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=PdUZT6ctqmI! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Error getting transcript for video TzIm6ecANW8: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=TzIm6ecANW8! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jde

In [19]:
# Display average word count by channel
average_word_count_by_channel = df_videos.groupby('channel_name')['word_count'].mean().sort_values(ascending=False)
print("\nAverage Word Count by Channel:")
print(average_word_count_by_channel)


Average Word Count by Channel:
channel_name
fplblackbox             23353.8
AboveAverageFPL         11916.2
TheArmbandFPL           10353.2
elitefpl                 8216.0
FFScout_                 6249.4
FPLFocal                 5067.6
fantasyfootballfixYT     3671.8
fplbanger                3378.2
Name: word_count, dtype: float64


In [20]:
df_videos.to_csv('transcripts.csv')

In [21]:
import spacy

# Load the English NLP model from spaCy
nlp = spacy.load('en_core_web_sm')

# Function to split transcript into manageable chunks based on token count
def split_into_chunks(transcript, max_tokens=4000):
    # Tokenize the transcript
    doc = nlp(transcript)
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in doc.sents:
        sentence_tokens = len(sentence.orth_.split())

        if current_tokens + sentence_tokens > max_tokens:
            # If the current chunk exceeds the limit, start a new one
            chunks.append(' '.join([str(sent) for sent in current_chunk]))
            current_chunk = [sentence.text]
            current_tokens = sentence_tokens
        else:
            current_chunk.append(sentence.text)
            current_tokens += sentence_tokens

    # Append any remaining chunk
    if current_chunk:
        chunks.append(' '.join([str(sent) for sent in current_chunk]))

    return chunks


In [26]:
# Function to process and chunk each transcript
def process_transcript(transcript, max_tokens=8000):
    if pd.isna(transcript):
        return []
    return split_into_chunks(transcript, max_tokens)


In [27]:
# Apply the chunking to each transcript in the DataFrame
df_videos['transcript_chunks'] = df_videos['transcript'].apply(process_transcript)